In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import gmaps


from api_keys import g_key

In [3]:
csv_file_path = "/Users/Jerry/Documents/city_weather.csv"
cities_df = pd.read_csv(csv_file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'city_weather.csv'

In [3]:
cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,busselton,-33.6500,115.3333,57.34,82,39,6.80,AU,1651507383
1,1,getulio vargas,-32.6796,-53.5494,64.89,56,52,7.49,UY,1651507384
2,2,verkhnechusovskiye gorodki,58.1671,57.1086,36.16,83,97,3.98,RU,1651507385
3,3,bluff,-46.6000,168.3333,46.80,81,46,8.66,NZ,1651507386
4,4,kodiak,57.7900,-152.4072,38.71,93,100,14.97,US,1651507387


In [4]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [16]:
# Heatmap of humidity
locations = cities_df[["Lat", "Lng"]]

humidity = cities_df["Humidity"]

map_fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

map_fig.add_layer(heat_layer)

map_fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.
#A max temperature lower than 80 degrees but higher than 70.
vacation_df = cities_df.loc[(cities_df["Max Temp"] >= 70 ) &
                           (cities_df["Max Temp"] < 90 ), :]

#Wind speed less than 10 mph.
vacation_df = vacation_df.loc[(vacation_df["Wind Speed"] < 10), :]

#Zero cloudiness.
vacation_df = vacation_df.loc[(vacation_df["Cloudiness"] == 0), :]

#Humidity is less than 70
vacation_df = vacation_df.loc[(vacation_df["Humidity"] < 70), :]


In [7]:
#create new data frame to add hotel names to corresponding cities
new_df = vacation_df[["City", "Country", "Lat", "Lng"]]
new_df["Hotel Name"] = ""

#check new dataframe
new_df

/var/folders/7y/crgpl7c16kz6hr5b9zbdn3vh0000gp/T/ipykernel_25066/2287966381.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Hotel Name
18,sao joao da barra,BR,-21.6403,-41.0511,
75,dehloran,IR,32.6941,47.2679,
115,riyadh,SA,24.6877,46.7219,
123,alghero,IT,40.5589,8.3181,
154,chanasma,IN,23.7167,72.1167,
247,mar de espanha,BR,-21.8672,-43.0097,
283,aranos,NaN,-24.1333,19.1167,
361,port hedland,AU,-20.3167,118.5667,
434,largo,US,27.9095,-82.7873,
488,pimentel,PE,-6.8367,-79.9342,


In [24]:
city_names = [new_df["City"]]
type(city_names)

list

In [38]:
#Test code for just one city

#get coordinates to city
city_1 = "sao joao da barra"
coord_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={city_1}&key={g_key}"
response1 = requests.get(coord_url).json()
lat1 = response["results"][0]["geometry"]["location"]["lat"]
lng1 = response["results"][0]["geometry"]["location"]["lng"]

base_url1 = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#create variables
target_type1 = "lodging"
radius1 = 5000

#create params
params1 = {
    "location": f"{lat1},{lng1}",
    "type": target_type1,
    "radius": radius1,
    "key": g_key
         }

#create API request
city_hotel_response = requests.get(base_url, params1)
city_hotel_json = city_hotel_response.json()

#extract hotel name and address
(city_hotel_json["results"][0]["name"])

'Pousada Amiga Márcia'

In [34]:
#first try at getting hotel names near city locations

#Use Google Places API to find the first hotel for each city located within 5,000 meters 
#of your coordinates.

for city in city_names:
    try:
        #get coordinates to citites
        city_name = city
        coord_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={city_name}&key={g_key}"
        response = requests.get(coord_url).json()
        lat2 = response["results"][0]["geometry"]["location"]["lat"]
        lng2 = response["results"][0]["geometry"]["location"]["lng"]

        base_url2 = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        #create variables
        target_type2 = "lodging"
        radius2 = 5000

        #create params
        params2 = {
            "location": f"{lat2},{lng2}",
            "type": target_type2,
            "radius": radius2,
            "key": g_key
                 }

        #create API request
        city_hotel_response = requests.get(base_url2, params2)
        #format in json
        city_hotel_json = city_hotel_response.json()

        #extract hotel name and address
        hotel_name = (city_hotel_json["results"][0]["name"])
    
        #append hotel name to dataframe corresponding to location
        hotels = []
        hotels.append(hotel_name)
    except:
        print("oops no information...skipping")


In [8]:
#url for place api
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#create variables
target_type = "lodging"
radius = 5000

# Iterate through cities and obtain lat and lng for api request
for index, row in new_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    #create params
    params = {
        "location": f"{lat},{lng}",
        "radius": radius,
        "types": target_type,
        "key": g_key}
    
    #create api request
    hotel_response = requests.get(base_url, params)
    
    #convert to json()
    hotel_json = hotel_response.json()
    
    #Get the first hotel append to corresponding city row
    try:
        new_df.loc[index, "Hotel Name"] = hotel_json["results"][0]["name"]
    except:
        print("Oops no information.. skipping")
    

/Users/Jerry/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [9]:
new_df

,City,Country,Lat,Lng,Hotel Name
18,sao joao da barra,BR,-21.6403,-41.0511,Pousada Porto De Canoas
75,dehloran,IR,32.6941,47.2679,کافه گیم استورم
115,riyadh,SA,24.6877,46.7219,"InterContinental Riyadh, an IHG Hotel"
123,alghero,IT,40.5589,8.3181,Smy Carlos V Alghero
154,chanasma,IN,23.7167,72.1167,Shree Bhateva Bhavan Dharamshala
247,mar de espanha,BR,-21.8672,-43.0097,Pousada Ayuruoca
283,aranos,NaN,-24.1333,19.1167,Ley-Lia Guest House
361,port hedland,AU,-20.3167,118.5667,The Esplanade Hotel
434,largo,US,27.9095,-82.7873,Hampton Inn & Suites Largo
488,pimentel,PE,-6.8367,-79.9342,Hotel Puerto del Sol


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in new_df.iterrows()]
locations = new_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
map_fig.add_layer(marker_layer)

# Display figure
map_fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
map_fig

Figure(layout=FigureLayout(height='420px'))